In [ ]:
import pandas as pd
import numpy as np
import re
import os

from google.colab import drive
drive.mount('/content/drive')

from A11_init_ import * # contains CLOVA id, secret key, url

In [94]:
## Load data
comments_df = pd.read_csv("./outputs/A10_comments_raw.csv")          # all comments
comments_df2 = pd.read_csv("./outputs/A10_comments_remove_dups.csv") # comments after eliminating entries by duplicated IDs
print(comments_df.shape)
print(comments_df2.shape)
comments_df2.loc[0:10,"comment"]

(4136, 2)
(985, 2)


0                        그놈의 vs질 참 지겹다왕석현100명vs뱅갈호랑이1마리
1                                구름위로 이건 석현이 의견도 들어봐야..
2                               구름위로 김성찬150명 vs 왕석현100명
3                     2000년생 이상 급식은 장범준90년대생이하는 성시경 나올듯
4     비정상인 반대임ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ00년생은 성시경 80년생은 장범준인 분위기였음ㅋㅋ...
5     비정상인 성시경 전성기를 모르니까 풍문으로 들은 애들이 더 고평가엠팍은 나이대가 높...
6                                              래드키 이게맞다
7                       래드키 이말이맞음 성시경이 00년대 goat 은 아니자너
8     델카이저 근데 장범준이 20년대 goat도 아니잖아 ;가창력은 성시경이 우위고,히트...
9                               원베일리 상업성 고려하면 장범준이 맞겠네.
10                    저교치기 GOAT는 한명이 맞음. 애초에 개념자체가 한명임.
Name: comment, dtype: object

In [38]:
## Split sung's and jang's dfs using keywords for each artists.
## Keywords incldue: name, nickname, titles of top hit songs.

sung_keywords = ["시경", "성식", "거리에서", "두사람", "희재"] 
jang_keywords = ["범준", "버스커", "벚꽃", "여수", "노래방에서"]

sung_df = pd.DataFrame(columns = ["ID", "comments"])
jang_df = pd.DataFrame(columns = ["ID", "comments"])
neutral_df = pd.DataFrame(columns = ["ID", "comments"])

for idx_c in range(0, len(comments_df2)):
    
    '''
    Each DF contains sentences that include any keyword for one artist, but no keyword for the other.
    If keywords from neither of the artists were included, then the sentence is stored in neutral_df.
    '''

    temp_id, temp_comment = comments_df2.iloc[idx_c]
    
    if any(sk in temp_comment for sk in sung_keywords) == True and any(jk in temp_comment for jk in jang_keywords) == False:
        sung_df.loc[len(sung_df)] = [temp_id, temp_comment]
    elif any(jk in temp_comment for jk in jang_keywords) == True and any(sk in temp_comment for sk in sung_keywords) == False:
        jang_df.loc[len(jang_df)] = [temp_id, temp_comment]
    else:
        neutral_df.loc[len(neutral_df)] = [temp_id, temp_comment]

In [41]:
## save dfs
sung_df.to_csv("./outputs/B10_sung_df.csv", index = False)
jang_df.to_csv("./outputs/B10_jang_df.csv", index = False)
neutral_df.to_csv("./outputs/B10_neutral_df.csv", index = False)

In [ ]:
## Perform sentiment analysis using CLOVA Sentiment API

import sys
import requests
import json

client_id = clova_id
client_secret = clova_secret

url = clova_api_url

headers = {
    "X-NCP-APIGW-API-KEY-ID": client_id,
    "X-NCP-APIGW-API-KEY": client_secret,
    "Content-Type": "application/json"
}

dfs = ["sung_df", "jang_df"]

### Merge the comments into one document.
### Sentences are separated by "."

for df in dfs: # for each of sung and jang's data frame do:
    
    result_df = pd.DataFrame(columns = ["content", "sentiment", "negative", "neutral", "positive", "highlight"])

    dat = "./outputs/B10_" + df + ".csv"
    load_dat = pd.read_csv(dat)

    n = 0
    m = 999

    ## Merge lines to create a document. Lines are separated by "."
    document = ".".join(load_dat.comments)
    document = document + "."
    max_doc_length = len(document)

    while n < max_doc_length:
        
        m = m if m <= max_doc_length else max_doc_length
        
        content = document[n:m] # Cut number of lines to max 1000 words (Limit set by CLOVA Sentiment API)
        while content[-1] != ".":
            content = content[0:len(content)-1]  # cut the document upto last "."

        ### Run sentiment analysis
        data = {
            "content": content
        }

        # print(json.dumps(data, indent = 4, sort_keys = True))
        response = requests.post(url, data = json.dumps(data), headers = headers)
        rescode = response.status_code
        
        if (rescode == 200):
            print(response.text)
        else:
            print("Error : " + response.text)

        text = response.json()

        ## Extract features from results
        for i in range(0, len(text['sentences'])):
            c = text["sentences"][i]["content"]
            sent = text["sentences"][i]["sentiment"]
            sent_neg = text["sentences"][i]['confidence']['negative']
            sent_neu = text["sentences"][i]['confidence']['neutral']
            sent_pos = text["sentences"][i]['confidence']['positive']
            offset = text["sentences"][i]['highlights'][0]["offset"]
            length = text["sentences"][i]['highlights'][0]["length"]
            highlight = text['sentences'][i]["content"][offset: offset + length]

            result_df.loc[len(result_df), ] = [c, sent, sent_neg, sent_neu, sent_pos, highlight]

        ## set new start/end points for next set of words.
        n = n + len(content)
        m = n + 999 if n + 999 <= max_doc_length else max_doc_length

    out_name = "./outputs/C10_" + df + "_sentiments.csv"
    result_df.to_csv(out_name, index = False)

In [54]:
sung_result = pd.read_csv("./outputs/C10_sung_df_sentiments.csv")
jang_result = pd.read_csv("./outputs/C10_jang_df_sentiments.csv")

In [74]:
jang_pos = sum(jang_result["positive"])
jang_neg = sum(jang_result["negative"])
jang_score = round(jang_pos / (jang_pos + jang_neg) * 100, 2)
print("Jang's score: {jang_score}".format(jang_score = jang_score))

Jang's score: 47.2


In [75]:
sung_pos = sum(sung_result["positive"])
sung_neg = sum(sung_result["negative"])
sung_score = round(sung_pos / (sung_pos + sung_neg) * 100, 2)
print("Sung's score: {sung_score}".format(sung_score = sung_score))

Sung's score: 42.68


In [95]:
sung_result.loc[sung_result["sentiment"] == "negative", ].head(10)

,content,sentiment,negative,neutral,positive,highlight
2,성시경은 고트급인 발라더들은 거론할 것도 없고 김종국도 못이김..,negative,0.968238,0.030567,0.001195,성시경은 고트급인 발라더들은 거론할 것도 없고
7,근데 생각해보면 엄청 터진 역대급 노래는 없는거 같기도하고?.,negative,0.992282,0.006071,0.001647,생각해보면 엄청 터진 역대급 노래는 없는거 같기도하고
20,무협초보 성시경 데뷔년도나 보고와서 이런말해라 ㅅㅂ ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ.,negative,0.997427,0.002334,0.000239,이런말해라 ㅅㅂ
22,어그로에많관부 원히터라기엔 성시경보다 오히려 히트곡은 더많을걸.,negative,0.997568,0.001913,0.000519,어그로에많관부 원히터라기엔 성시경보다 오히려 히트곡은 더많을걸
25,곡만 받아서 부른다는게 외려 편견임..,negative,0.997889,0.001472,0.000639,곡만 받아서 부른다는게 외려 편견임
26,성시경 노래는 좋았는데 걍 임팩트가 없었음 ㅋㅋ,negative,0.995230,0.001143,0.003627,임팩트가 없었음
28,당시 최고히트곡이였던 거리에서보다 이기찬 감기 이런게 더 순위 높았을껄.,negative,0.992110,0.004683,0.003207,거리에서보다 이기찬 감기 이런게 더 순위 높았을껄
31,이런 반응이라 더 냉철한듯여기는 아직도 롤보다 스타라는 사람들도 있어서 버즈보다 ...,negative,0.995054,0.003670,0.001276,사람들고 있었고 객관적이지 못함
32,다이나믹아일랜드 진짜 조금만 파고들어도 성시경 테크닉이 진짜 엄청나게 어려운건데 노...,negative,0.884025,0.103823,0.012152,성시경 테크닉이 진짜 엄청나게 어려운건데 노래를 너무 쉽게 부르는 것처럼 보여지니
33,성시경이 존나게 위지 뭔 개소리들을 쳐하고 자빠졌냐,negative,0.999202,0.000432,0.000366,위지 뭔 개소리들을 쳐하고


In [96]:
jang_result.loc[jang_result["sentiment"] == "negative", ].head(10)

,content,sentiment,negative,neutral,positive,highlight
15,"애초에 비교불가, 장범준이 넘사임.",negative,0.991874,0.007511,0.000615,장범준이 넘사임
25,그러나 버스커 버스커 앨범과는 다른 실험적이고 마이너한 감성이었던 1집은 대중들에...,negative,0.997187,0.002453,0.000360,대중들에게 큰 어필을 하지 못했다
28,2017년 05월 15일 군 입대를 앞두고 마지막으로 낸 음원이었지만 장범준 스스...,negative,0.998680,0.000740,0.000579,말할 정도로 상업적으로 가장 부진하였다
29,이런 애들은 장범준 앨범도 꽤 망한 게 많은데 모르나보네ㅋㅋㅋ,negative,0.998877,0.000646,0.000477,이런 애들은 장범준 앨범도 꽤 망한 게 많은데
33,허밋 히트곡도 장범준이 딱히 안밀림.,negative,0.988480,0.010356,0.001164,허밋 히트곡도 장범준이 딱히 안밀림
34,섹보스지스털섹 공감 ㅋㅋ,negative,0.996454,0.003125,0.000421,섹보스지스털섹 공감
38,노하라히로시 벚꽃엔딩이면 평정이지.,negative,0.978946,0.018140,0.002914,노하라히로시 벚꽃엔딩이면 평정이지
43,걍 취향이지 우위가어딧냐 시발 ㅋㅋ,negative,0.991849,0.007695,0.000455,취향이지 우위가어딧냐 시발
47,애초에 장범준은 다른사람한테 곡을 못 받음자기한테 맞춰서 노래를 기가맥히게 만드는거지.,negative,0.992478,0.006163,0.001359,노래를 기가맥히게 만드는거지
48,노하라히로시 장범준: 뭐야 나 왜죽여요.,negative,0.996766,0.002844,0.000390,나 왜죽여요


In [ ]:
### Add word clouds